# Test Modules

In [4]:
# Set project root directory and add `src` to path
import sys
from pathlib import Path

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import the notebook setup utilities
from word2gm_fast.utils.notebook_setup import setup_testing_notebook, enable_autoreload

# Enable autoreload for development
enable_autoreload()

# Set up environment (GPU-enabled for tests)
env = setup_testing_notebook(project_root=PROJECT_ROOT)

# Extract commonly used modules for convenience
tf = env['tensorflow']
np = env['numpy']
pd = env['pandas']
batch_prepare_training_data = env['batch_prepare_training_data']
print_resource_summary = env['print_resource_summary']

ImportError: cannot import name 'setup_testing_notebook' from 'word2gm_fast.utils.notebook_setup' (/scratch/edk202/word2gm-fast/src/word2gm_fast/utils/notebook_setup.py)

In [ ]:
# Clean test execution - no duplicates
print("🧪 Testing all modules...")

modules_to_test = [
    ('tests.test_corpus_to_dataset', 'Corpus processing'),
    ('tests.test_index_vocab', 'Vocabulary indexing'), 
    ('tests.test_dataset_to_triplets', 'Triplet generation'),
    ('tests.test_tfrecord_io', 'TFRecord I/O')
]

test_results = []
for test_module, test_name in modules_to_test:
    if 'triplets' in test_module:
        cmd_result = run_silent_subprocess(
            ['python', '-m', 'unittest', '-b', test_module], 
            deterministic=False, capture_output=True, text=True
        )
    else:
        cmd_result = run_silent_subprocess(
            ['python', '-m', 'unittest', '-b', test_module], 
            capture_output=True, text=True
        )
    
    status = "PASS" if cmd_result.returncode == 0 else "FAIL"
    print(f"  {test_name}: {status}")
    test_results.append(cmd_result.returncode == 0)

passes = sum(test_results)
total = len(test_results)
print(f"\nResult: {passes}/{total} tests passed")
if passes == total:
    print("✅ All tests successful!")
else:
    print("❌ Some tests failed")